In [ ]:
'''
on the jupyter notebook launch terminal install--
pip install mediapipe opencv-python numpy
'''

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time # time: Can be used for FPS calculations (not used in this version).

# Load MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False, 
    max_num_faces=2, 
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
'''
mp_face_mesh.FaceMesh(...): Creates a face mesh model with:
static_image_mode=False: Optimized for video (tracking instead of detecting each frame separately).
max_num_faces=2: Detect up to 2 faces at a time.
min_detection_confidence=0.5: 50% confidence threshold to detect a face.
min_tracking_confidence=0.5: 50% confidence threshold to track landmarks.
mp_draw = mp.solutions.drawing_utils: Loads functions to draw face landmarks.

'''
'''
# Load video file
video_path = "video.mp4"  # Replace with your video file
cap = cv2.VideoCapture(video_path)
'''
cap = cv2.VideoCapture(0) 
# Check if video file is opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    cap.release()
else:
    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    '''
    Gets the video’s width, height, and FPS.
    Creates a VideoWriter object (out) to save the processed video.
    '''

    # Define video writer to save output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("output.mp4", fourcc, fps, (frame_width, frame_height))

    print("Press 'q' to exit the video window.")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit loop when video ends

        # Convert BGR to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process frame with Face Mesh
        results = face_mesh.process(rgb_frame)
        '''
        Reads a frame from the video (cap.read()).
        If the frame is empty (end of video), it exits the loop.
        Converts the BGR frame to RGB (because MediaPipe uses RGB).
        Processes the frame to detect face mesh landmarks.
        '''

        # Draw landmarks if face is detected
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_draw.draw_landmarks(
                    frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,
                    mp_draw.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                    mp_draw.DrawingSpec(color=(255, 255, 255), thickness=1)
                )
        '''
        Checks if any face was detected (if results.multi_face_landmarks:).
        Loops through all detected faces.
        Draws landmarks on the face, connecting 468 key points.
        Green dots (circle_radius=1) for keypoints and white lines (thickness=1) for connections.
        '''

        # Show the frame in a separate OpenCV window
        cv2.imshow("Face Mesh Output", frame)

        # Press 'q' to exit the video window
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Write processed frame to output file
        out.write(frame)
        '''
        Displays the processed frame in a separate OpenCV window.
        Waits for keypress ('q') to exit (cv2.waitKey(1) & 0xFF == ord('q')).
        Writes the processed frame to "output.mp4".

        '''

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("Video processing complete and window closed.")

'''
Releases the video and output file resources.
Closes the OpenCV window (cv2.destroyAllWindows()).

'''

🎥 Press 'q' to exit the video window.
✅ Video processing complete and window closed.


In [ ]:
# Holistic

In [3]:
# Initialize the models

mp_holistic = mp.solutions.holistic

holistic_model = mp_holistic.Holistic()

mp_drawing = mp.solutions.drawing_utils

In [9]:
# Capture the video

capture = cv2.VideoCapture(0)

# For calculation of Frame Per Second (FPS)

previousTime = 0
currentTime = 0

while capture.isOpened():

    # Capture the frame
    ret, frame = capture.read()

    # Resize the frame
    frame = cv2.resize(frame, (800,600))

    # Convert from BGR to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect Landmarks
    image.flags.writeable = False
    results = holistic_model.process(image)
    image.flags.writeable = True

    # Convert back to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Drawing Facial Landmarks

    mp_drawing.draw_landmarks(
    image, results.face_landmarks,
    mp_holistic.FACEMESH_CONTOURS,
    mp_drawing.DrawingSpec(
    color = (255, 0, 255),
    thickness = 1,
    circle_radius = 1),
    mp_drawing.DrawingSpec(
    color = (255,0,255),
    thickness=1,
    circle_radius=1))

    # Drawing Right Hand Landmarks

    mp_drawing.draw_landmarks(
    image, results.right_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS)

    # Drawing Left Hand Landmarks

    mp_drawing.draw_landmarks(
    image, results.left_hand_landmarks,
    mp_holistic.HAND_CONNECTIONS)

    # Calculate the FPS
    currentTime = time.time()
    fps = 1/(currentTime-previousTime)
    previousTime = currentTime

    # Displaying FPS on the image

    cv2.putText(image,
                str(int(fps))+" FPS",
                (10,70),
                cv2.FONT_HERSHEY_COMPLEX,
                1, (0, 255, 0),
                2)

    # Show the frame
    cv2.imshow('Mediapipe Practical', image)
    key = cv2.waitKey(5)
    if key == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
# Pose Detection